# Reasoning Experiments

In [9]:
# %pip install langchain langchain-core langchain-community langchain-openai
# %pip install --force-reinstall typing-extensions==4.5
# %pip install --force-reinstall openai==1.8

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are us

## Load Libraries

In [1]:
import os
import sys
import pandas as pd
sys.path.append('..')
from src.prompt import PROMPTS
from src.evaluation import Evalator
from src.helpers import fix_decision_parser
from src.experiment_runner import run_experiment

from huggingface_hub import login
from datasets import Dataset, DatasetDict

pd.set_option('display.max_colwidth', None)

login(token=os.environ.get("HF_TOKEN"), add_to_git_credential=True)

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/nazardrushchak/.cache/huggingface/token
Login successful


## Load Data

In [2]:
df_uk = pd.read_csv('../data/uk_data_samples.csv')
df_en = pd.read_csv('../data/en_data_samples.csv')

## `gpt-3.5-turbo-0125`: English Language experiment

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [4]:
chain = PROMPTS['reasoning_en'] | llm

In [5]:
# # DATA FOR TEST. Comment for real run
# test_id = ["dcd1541d-010b-5fbc-a0af-acc555d59b34_0ee38dd2-01cd-55b1-bb5b-e65aec4db7d9", "4f3a8628-3c36-5636-b22a-96d75fda88dd_ce7217d0-756c-5928-859a-e12911bd157d"]
# df_en = df_en[df_en['item_id'].isin(test_id)]
# df_en

In [7]:
file_paths = run_experiment(
    folder_path='../data/reasoning',
    chain=chain,
    data=df_en,
    lang='en',
    batch_size=5,
)

In [8]:
file_paths

{'gender': '../data/reasoning/en/gender.csv',
 'marital_status': '../data/reasoning/en/marital_status.csv',
 'military_status': '../data/reasoning/en/military_status.csv',
 'religion': '../data/reasoning/en/religion.csv',
 'name': '../data/reasoning/en/name.csv',
 'age': '../data/reasoning/en/age.csv'}

## Search best prompt for the experiment

In [10]:
# # ONLY FOR TEST. Comment for real run
# FILES_PATHS = {
#     'gender': '../data/reasoning/en/gender.csv',
#     'marital_status': '../data/reasoning/en/marital_status.csv',
#     'military_status': '../data/reasoning/en/military_status.csv',
#     'religion': '../data/reasoning/en/religion.csv',
#     'name': '../data/reasoning/en/name.csv',
#     'age': '../data/reasoning/en/age.csv'
#     }

# dataset_dict = {}
# for key_name, file_path in FILES_PATHS.items():
#     df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
#     df_part['decision'] = df_part['decision'].map(lambda x: x.lower())

#     print(key_name)
#     for exmaple_num, group_id in enumerate(df_part['group_id'].unique()):
#         df_group = df_part[df_part['group_id'] == group_id]
#         #print(df_group['decision'].value_counts())
#         print(f"Example {exmaple_num + 1} Consistency:", df_group['decision'].value_counts().max() / df_group['decision'].value_counts().sum())
#     print()

gender
Example 1 Consistency: 0.7272727272727273
Example 2 Consistency: 0.9090909090909091

marital_status
Example 1 Consistency: 1.0
Example 2 Consistency: 0.6

military_status
Example 1 Consistency: 0.8
Example 2 Consistency: 0.6

religion
Example 1 Consistency: 0.5555555555555556
Example 2 Consistency: 0.6666666666666666

name
Example 1 Consistency: 1.0
Example 2 Consistency: 0.7

age
Example 1 Consistency: 0.6666666666666666
Example 2 Consistency: 0.8333333333333334



## Load English Results to HF Datasets

In [2]:
FILES_PATHS = {
    'gender': '../data/reasoning/en/gender.csv',
    'marital_status': '../data/reasoning/en/marital_status.csv',
    'military_status': '../data/reasoning/en/military_status.csv',
    'religion': '../data/reasoning/en/religion.csv',
    'name': '../data/reasoning/en/name.csv',
    'age': '../data/reasoning/en/age.csv'
    }

# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
    df_part['decision'] = df_part['decision'].map(lambda x: x.lower())
    df_part = df_part.groupby(by=['candidate_id', 'job_id','CV','Job Description', 'Job Position', 'lang', 'protected_group', 'protected_attr', 'group_id']).agg({
        "decision": "first",
        "feedback": "first",
        "raw_ai_decision": "first",
    }).reset_index()
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-reasoning-en', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-reasoning-en/commit/b1b5e428c93684fb33e18960eca049e0f6e1d9b6', commit_message='Upload dataset', commit_description='', oid='b1b5e428c93684fb33e18960eca049e0f6e1d9b6', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-reasoning-en", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [4]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-reasoning-en", "reasoning")

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Generating gender split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [5]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,reasoning,gender,en,0.7913,0.9653,1.0,"{'Agender': 0.6311, 'Androgynous': 0.6089, 'Bigender': 0.54, 'Cisgender': 0.6067, 'Demiboy': 0.5844, 'Demigender': 0.52, 'Demigirl': 0.6044, 'Female': 0.6156, 'Gender Nonconforming': 0.5867, 'Genderfluid': 0.58, 'Genderqueer': 0.5911, 'Intersex': 0.5644, 'Male': 0.5422, 'Neutrois': 0.6, 'Non-Binary': 0.6311, 'Pangender': 0.5311, 'Queer': 0.58, 'Third Gender': 0.5244, 'Transgender': 0.6022, 'Two-Spirit': 0.5244}","{'Agender': 0.0622, 'Androgynous': 0.0533, 'Bigender': 0.0556, 'Cisgender': 0.0511, 'Demiboy': 0.0378, 'Demigender': 0.0622, 'Demigirl': 0.0489, 'Female': 0.0644, 'Gender Nonconforming': 0.0711, 'Genderfluid': 0.0511, 'Genderqueer': 0.04, 'Intersex': 0.0444, 'Male': 0.0622, 'Neutrois': 0.0489, 'Non-Binary': 0.0756, 'Pangender': 0.06, 'Queer': 0.0422, 'Third Gender': 0.0622, 'Transgender': 0.0556, 'Two-Spirit': 0.0844}"
1,reasoning,marital_status,en,0.8074,0.9646,1.0,"{'Civil union': 0.5267, 'Divorced (Divorced)': 0.5489, 'Married (Husband/Wife)': 0.54, 'Unmarried (Single)': 0.5689, 'Widower (Widow)': 0.5356}","{'Civil union': 0.0467, 'Divorced (Divorced)': 0.0467, 'Married (Husband/Wife)': 0.0556, 'Unmarried (Single)': 0.04, 'Widower (Widow)': 0.0556}"
2,reasoning,military_status,en,0.7777,0.9568,1.0,"{'Civilian': 0.5956, 'Military retiree': 0.5178, 'Participant in combat actions': 0.3333, 'Reservist': 0.5711, 'War veteran': 0.5444}","{'Civilian': 0.0778, 'Military retiree': 0.04, 'Participant in combat actions': 0.2022, 'Reservist': 0.0533, 'War veteran': 0.0489}"
3,reasoning,religion,en,0.7885,0.9665,1.0,"{'atheist': 0.6111, 'buddhist': 0.4444, 'christian': 0.5578, 'hindu': 0.4733, 'jain': 0.4844, 'jew': 0.4756, 'muslim': 0.4689, 'sikh': 0.48, 'zoroastrian': 0.44}","{'atheist': 0.12, 'buddhist': 0.0733, 'christian': 0.0756, 'hindu': 0.0622, 'jain': 0.0556, 'jew': 0.06, 'muslim': 0.08, 'sikh': 0.0556, 'zoroastrian': 0.0644}"
4,reasoning,name,en,0.7632,0.9160,1.0,"{'Aaron': 0.5889, 'Amartol': 0.5622, 'Emma': 0.5933, 'Khulian': 0.5667, 'Liusia': 0.5756, 'Liusiia': 0.5889, 'Romchyk': 0.5622, 'Siu': 0.5422, 'Tyhran': 0.58, 'Zenoviia': 0.5911}","{'Aaron': 0.0556, 'Amartol': 0.0244, 'Emma': 0.0422, 'Khulian': 0.0244, 'Liusia': 0.0333, 'Liusiia': 0.0289, 'Romchyk': 0.0467, 'Siu': 0.0311, 'Tyhran': 0.0378, 'Zenoviia': 0.0578}"
5,reasoning,age,en,0.7856,0.9464,1.0,"{'20': 0.4778, '30': 0.5822, '40': 0.4511, '50': 0.2844, '60': 0.14, '70': 0.0533}","{'20': 0.2022, '30': 0.3067, '40': 0.1756, '50': 0.0089, '60': 0.1356, '70': 0.2222}"


In [6]:
EVALUATION_FILE  = '../data/evaluation_results_en.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_en.csv


## `gpt-3.5-turbo-0125`: Ukrainian Language experiment

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [4]:
chain = PROMPTS['reasoning_uk'] | llm

In [5]:
# # DATA FOR TEST. Comment for real run
# test_id = ["e765137d-40e0-5ae0-aaf9-a966f76f3621_13dbe82c-a0fa-5538-82a6-64f8268ece38", "09596984-6d06-5d6c-81ef-ab79203cf4c6_fa4a8421-b9ac-50e2-a83b-056e62698359"]
# df_uk = df_uk[df_uk['item_id'].isin(test_id)]
# df_uk

In [6]:
file_paths = run_experiment(
    folder_path='../data/reasoning',
    chain=chain,
    data=df_uk,
    lang='uk',
    batch_size=25,
)

In [7]:
file_paths

{'gender': '../data/reasoning/uk/gender.csv',
 'marital_status': '../data/reasoning/uk/marital_status.csv',
 'military_status': '../data/reasoning/uk/military_status.csv',
 'religion': '../data/reasoning/uk/religion.csv',
 'name': '../data/reasoning/uk/name.csv',
 'age': '../data/reasoning/uk/age.csv'}

## Search best prompt for the experiment

In [8]:
# # ONLY FOR TEST. Comment for real run
# FILES_PATHS = {
#     'gender': '../data/reasoning/uk/gender.csv',
#     'marital_status': '../data/reasoning/uk/marital_status.csv',
#     'military_status': '../data/reasoning/uk/military_status.csv',
#     'religion': '../data/reasoning/uk/religion.csv',
#     'name': '../data/reasoning/uk/name.csv',
#     'age': '../data/reasoning/uk/age.csv'
#     }

# def ukr_decision_fixing(x: str) -> str:
#     if not x:
#         return x
#     for word in x.split():
#         for part in ['найн', 'наїн', 'наєн', 'прий', 'accept', 'hire']:
#             if word.lower().startswith(part):
#                 return 'hire'
#         for part in ['відхил', 'reject', 'відмов', 'вибачте']:
#             if word.lower().startswith(part):
#                 return 'reject'
#     return x

# dataset_dict = {}
# for key_name, file_path in FILES_PATHS.items():
#     df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
#     df_part['decision'] = df_part['decision'].map(ukr_decision_fixing)

#     print(key_name)
#     for exmaple_num, group_id in enumerate(df_part['group_id'].unique()):
#         df_group = df_part[df_part['group_id'] == group_id]
#         #print(df_group['decision'].value_counts())
#         print(f"Example {exmaple_num + 1} Consistency:", df_group['decision'].value_counts().max() / df_group['decision'].value_counts().sum())
#     print()

gender
Example 1 Consistency: 0.8181818181818182
Example 2 Consistency: 0.5909090909090909

marital_status
Example 1 Consistency: 1.0
Example 2 Consistency: 1.0

military_status
Example 1 Consistency: 0.8
Example 2 Consistency: 1.0

religion
Example 1 Consistency: 0.7777777777777778
Example 2 Consistency: 1.0

name
Example 1 Consistency: 0.9
Example 2 Consistency: 0.7

age
Example 1 Consistency: 0.6666666666666666
Example 2 Consistency: 1.0



## Load Ukrainian Results to HF Datasets

In [7]:
FILES_PATHS = {
    'gender': '../data/reasoning/uk/gender.csv',
    'marital_status': '../data/reasoning/uk/marital_status.csv',
    'military_status': '../data/reasoning/uk/military_status.csv',
    'religion': '../data/reasoning/uk/religion.csv',
    'name': '../data/reasoning/uk/name.csv',
    'age': '../data/reasoning/uk/age.csv'
    }

def ukr_decision_fixing(x: str) -> str:
    if not x:
        return x
    for word in x.split():
        for part in ['найн', 'наїн', 'наєн', 'прий', 'accept', 'hire']:
            if word.lower().startswith(part):
                return 'hire'
        for part in ['відхил', 'reject', 'відмов', 'вибачте']:
            if word.lower().startswith(part):
                return 'reject'
    return x


# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})

    df_part = fix_decision_parser(df_part)
    df_part['decision'] = df_part['decision'].map(ukr_decision_fixing)
    df_part = df_part.groupby(by=['candidate_id', 'job_id','CV','Job Description', 'Job Position', 'lang', 'protected_group', 'protected_attr', 'group_id']).agg({
        "decision": "first",
        "feedback": "first",
        "raw_ai_decision": "first",
    }).reset_index()
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-reasoning-uk', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-reasoning-uk/commit/4e18d95e837207c468aa36cdc6eb590c7dc0dfeb', commit_message='Upload dataset', commit_description='', oid='4e18d95e837207c468aa36cdc6eb590c7dc0dfeb', pr_url=None, pr_revision=None, pr_num=None)

In [8]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-reasoning-uk", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [9]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-reasoning-uk", "reasoning")

Generating gender split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [10]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,reasoning,gender,uk,0.8376,0.9549,1.0,"{'Інтерсекс': 0.2333, 'Агендер': 0.1844, 'Андрогінний': 0.2133, 'Бігендер': 0.2111, 'Гендерквір': 0.2311, 'Гендерне невідповідність': 0.0511, 'Гендерфлюїд': 0.2133, 'Дводушний (Твоуспірит)': 0.3244, 'Демігендер': 0.1978, 'Демідівчина': 0.2733, 'Деміхлопчик': 0.1622, 'Квір': 0.1489, 'Небінарний': 0.3133, 'Неутроїс': 0.1978, 'Пангендер': 0.2156, 'Трансгендер': 0.12, 'Третя стать': 0.1578, 'Цісгендер': 0.2489, 'Чоловік': 0.1889, 'Жінка': 0.2378}","{'Інтерсекс': 0.1111, 'Агендер': 0.1111, 'Андрогінний': 0.0956, 'Бігендер': 0.0933, 'Гендерквір': 0.1089, 'Гендерне невідповідність': 0.1244, 'Гендерфлюїд': 0.0956, 'Дводушний (Твоуспірит)': 0.1933, 'Демігендер': 0.1111, 'Демідівчина': 0.1511, 'Деміхлопчик': 0.0889, 'Квір': 0.08, 'Небінарний': 0.1778, 'Неутроїс': 0.1111, 'Пангендер': 0.1289, 'Трансгендер': 0.0956, 'Третя стать': 0.1022, 'Цісгендер': 0.1444, 'Чоловік': 0.1244, 'Жінка': 0.1289}"
1,reasoning,marital_status,uk,0.8490,0.9565,1.0,"{'Вдовець/Вдова': 0.1778, 'Неодружений/Неодружена': 0.2556, 'Одружений/Одружена': 0.2956, 'Розлучений/Розлучена': 0.1867, 'Цивільний шлюб': 0.3044}","{'Вдовець/Вдова': 0.0911, 'Неодружений/Неодружена': 0.08, 'Одружений/Одружена': 0.1111, 'Розлучений/Розлучена': 0.0956, 'Цивільний шлюб': 0.1244}"
2,reasoning,military_status,uk,0.8337,0.9490,1.0,"{'Ветеран війни': 0.0889, 'Військовий пенсіонер': 0.0422, 'Резервіст': 0.1556, 'Учасник бойових дій': 0.0644, 'Цивільний': 0.1533}","{'Ветеран війни': 0.0333, 'Військовий пенсіонер': 0.0444, 'Резервіст': 0.1, 'Учасник бойових дій': 0.0267, 'Цивільний': 0.0889}"
3,reasoning,religion,uk,0.8415,0.9552,1.0,"{'атеїст': 0.1822, 'буддист': 0.1067, 'джайніст': 0.0978, 'зороастрист': 0.0867, 'мусульманин': 0.0533, 'сикх': 0.16, 'християнин': 0.1667, 'єврей': 0.1511, 'індуїст': 0.0978}","{'атеїст': 0.1067, 'буддист': 0.0533, 'джайніст': 0.0444, 'зороастрист': 0.0467, 'мусульманин': 0.0533, 'сикх': 0.1022, 'християнин': 0.0956, 'єврей': 0.0978, 'індуїст': 0.0578}"
4,reasoning,name,uk,0.8029,0.9496,1.0,"{'Аарон': 0.2622, 'Амартол': 0.2533, 'Емма': 0.3111, 'Зеновія': 0.2533, 'Люся': 0.3111, 'Люсія': 0.34, 'Ромчик': 0.2067, 'Сю': 0.2622, 'Тигран': 0.2578, 'Хуліан': 0.3}","{'Аарон': 0.0756, 'Амартол': 0.0889, 'Емма': 0.1244, 'Зеновія': 0.08, 'Люся': 0.1111, 'Люсія': 0.1444, 'Ромчик': 0.1089, 'Сю': 0.1022, 'Тигран': 0.1111, 'Хуліан': 0.1133}"
5,reasoning,age,uk,0.8236,0.9474,1.0,"{'20': 0.1911, '30': 0.2356, '40': 0.1089, '50': 0.0489, '60': 0.0156, '70': 0.0111}","{'20': 0.1533, '30': 0.1978, '40': 0.0711, '50': 0.02, '60': 0.0222, '70': 0.0267}"


In [11]:
EVALUATION_FILE  = '../data/evaluation_results_uk.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_uk.csv
